In [ ]:
import ast
import numpy as np
import pandas as pd

from opportunistic_planning import prediction, processing

In [ ]:
data_ts = pd.read_csv('all_task_environments_list_2022-06-02_unique_sequences.csv', header=0)

In [ ]:
data = pd.read_csv('kit_task_envs_2022-08-30.csv', header=0)

In [ ]:
distances_dict_kit = processing.generate_distances_dict(data, use_string_for_seq=False,
                                                   dimensions=[[2, 'xy']])

### fit model on tablesetting

In [ ]:
tablesetting_data = data[:186]
tablesetting_data.reset_index(inplace=True)

In [ ]:
distances_dict = processing.generate_distances_dict(data, use_string_for_seq=False,
                                                   dimensions=[[2, 'xy']])

In [ ]:
results_sum = processing.calculate_prediction_error(tablesetting_data, distances_dict, error_function='prequential', n=100, dimensions=[[2, 'xy']])

In [ ]:
lowest_mean, lowest_mean_idx, lowest_median, results_median = processing.get_lowest_error(results_sum)

In [ ]:
lowest_mean_idx

### test on kit cleaning data

In [ ]:
results = pd.DataFrame()

for row in range(0, len(data)):
        use_string_for_seq = False
        coordinates = {key: ast.literal_eval(value) for key, value in
                       (elem.split(': ') for elem in data.at[row, 'coordinates'].split(';'))}

        start_coordinates = list(ast.literal_eval(data.at[row, 'start_coordinates']))
        ID = str(data.at[row,'ID'])
        
        if use_string_for_seq == True:
            seq = str(data.at[row, 'sequence'])
            objects = list(data.at[row, 'sequence'])
        else:
            seq = [elem for elem in data.at[row, 'sequence'].split(',')]
            objects = [elem for elem in data.at[row, 'sequence'].split(',')]

        # get list of objects that have relational dependencies, if any (else set to empty list)
        try:
            strong_k = list(data.at[row, 'strong_k'].split(','))
        except AttributeError:
            strong_k = []

        try:
            mid_k = list(data.at[row, 'mid_k'].split(','))
        except AttributeError:
            mid_k = []

        try:
            food_k = list(data.at[row, 'food_k'].split(','))
        except AttributeError:
            food_k = []
            
        k1 = {obj: 1.7 if obj in food_k else 0.2 if obj in strong_k \
              else 0.3 if obj in mid_k else 1.0 for obj in objects}
        try:
            c1 = {obj: 1.8 if obj in data.at[row, 'containment'] else 1.0 for obj in objects}
        except:
            c1 = {obj: 1.0 for obj in objects}
            
        dim = [2, 'xy']
        
        median = processing.get_median_error('prequential', 
                                             row, ID, objects, coordinates, start_coordinates,
                                             c1, k1, dim, seq, distances_dict_kit, n=100)
        
        results.at[row, 'median'] = median
        results.at[row, 'median_normalized'] = median / len(seq)
        results.at[row, 'ID'] = ID

In [ ]:
results

In [ ]:
np.median(results['median'])

In [ ]:
results.to_csv('results/results_cleaning_2D_n100_fitted_to_ts_2023-01-20.csv', header=True, index=False)